In [3]:
from multiprocessing import Process, Queue
import multiprocessing
from datasets.arrow_dataset import Dataset
from transformers import PreTrainedTokenizer
from typing import List, Dict
import torch
import random
import tqdm
import glob
from pathlib import Path
from datasets import concatenate_datasets, Dataset
from transformers import AutoTokenizer
#from model import load_pretrained_bert_base
from transformers import AutoModelForSequenceClassification, AutoModelForMaskedLM,AutoModelForPreTraining
#from model import get_bert_config, BertForSequenceClassificationWithLoss, BertForMaskedLMWithLoss
from transformers import BertConfig, BertForMaskedLM


In [44]:
model1 = AutoModelForPreTraining.from_pretrained("../../models/general_tinybert_pretrain/")
model2 = AutoModelForPreTraining.from_pretrained("../../models/pretrained_bert_mlm/")
model3 = AutoModelForPreTraining.from_pretrained("prajjwal1/bert-small")

Some weights of BertForPreTraining were not initialized from the model checkpoint at ../../models/pretrained_bert_mlm/ and are newly initialized: ['bert.pooler.dense.weight', 'bert.pooler.dense.bias', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [45]:
tok = AutoTokenizer.from_pretrained("bert-base-uncased")

In [46]:
from transformers import BertTokenizer

In [61]:
b = tok(["If I lie in my bed for a long time I will fall [MASK] eventually."])
b = {k:torch.tensor(v) for k,v in b.items()}
print(tok.decode(model1(**b).prediction_logits.argsort(dim=-1)[0,0].tolist()[0]))
print(tok.decode(model2(**b).prediction_logits.argmax(dim=-1).tolist()[0]))
print(tok.decode(model3(**b).prediction_logits.argmax(dim=-1).tolist()[0]))

##logists
. if i lie in my bed for a long time i will fall asleep eventually..
will if i lie in my bed for a long time i will fall asleep eventually. fall


In [37]:
import tqdm
import json
import time
times = []
for _ in range(5):
    start = time.time()
    with open("test.txt","r") as f:
        data = []
        while True:
            line = f.readline()
            if line is None:
                continue
            try:
                data.append(json.loads(line))
            except:
                print(line)
                break
            if len(data)%1000==0:
                print(len(data))
    times.append(time.time()-start)

1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000
27000
28000
29000
30000
31000
32000
33000
34000
35000
36000
37000
38000
39000
40000
41000
42000
43000
44000
45000
46000
47000
48000
49000
50000
51000
52000
53000
54000
55000
56000
57000
58000
59000
60000
61000
62000
63000
64000
65000
66000
67000
68000
69000
70000
71000
72000
73000
74000
75000
76000
77000
78000
79000
80000
81000
82000
83000
84000
85000
86000
87000
88000
89000
90000
91000
92000
93000
94000
95000
96000
97000
98000
99000
100000
101000
102000
103000
104000
105000
106000
107000
108000
109000
110000
111000
112000
113000
114000
115000
116000
117000
118000
119000
120000
121000
122000
123000
124000
125000
126000
127000
128000
129000
130000
131000
132000
133000
134000
135000
136000
137000
138000
139000
140000
141000
142000
143000
144000
145000
146000
147000
148000
149000
150000
151000
152000
153000
154000
155000
156000
157000
158000
15

In [17]:
len(data)

685353

In [18]:
from datasets.arrow_dataset import Dataset

dataset = Dataset.from_list(data)

In [20]:
dataset.save_to_disk("dataset.arrow")

In [25]:
d2 = Dataset.load_from_disk("dataset.arrow")

In [26]:
d2.set_format("torch")

In [28]:
tokens = d2['tokens']

In [31]:
segment_ids = d2["segment_ids"]

In [35]:
import pickle
pickle.dump(data,open("dataset","wb"))

In [36]:
data2 = pickle.load(open("dataset", "rb"))

In [1]:
from kd import KDSequenceClassification

In [5]:
teacher = AutoModelForSequenceClassification.from_pretrained("bert-base-uncased")
student = AutoModelForSequenceClassification.from_pretrained("bert-base-uncased")

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

In [6]:
kdmodel = KDSequenceClassification(teacher,student,[])

In [7]:
torch.save(kdmodel,"../../tmp")

In [ ]:

    from load_glue import load_tokenized_glue_dataset


datasets = load_tokenized_glue_dataset("../../GLUE-baselines/glue_data/", "SST-2",augmented=False)